# Tests unitaires de l'API
## Scénario de test unitaire 
Vu qu'il s'agit d'un modèle de classification binaire, le test unitaire consistera à :
- faire prédire la variable cible par l'API du modèle 
- fournir au modèle les valeurs des attributs via un formulaire

Le micro-framework Web léger [_Flask_](https://flask.palletsprojects.com/en/3.0.x/quickstart/) prend en charge la gestion de requêtes et réponses REST

## API
Mise en service

## Interface Web de l'API
### Fichiers _HTML_

In [6]:
dir_html = './templates/'

#### Index  
Fichier d'accueil

In [7]:
file_html_index = dir_html + 'index.html'
!cat $file_html_index


<!DOCTYPE html>
<html lang="en">
    <head>
        <meta charset="UTF-8">
        <link rel="stylesheet" href="{{ url_for('static', filename= 'css/style.css') }}">
        <title>ML model API</title>
    </head>
    <body>
       <h1>API du modèle de classification binaire</h1>      
       <h2>Page d'accueil</h2>    
       <a href="/form/">Formulaire</a>
    </body>
</html>



#### Formulaire  
Formulaire de saisie des atttributs

In [8]:
file_html_form = dir_html + 'form.html'
!cat $file_html_form


<!DOCTYPE html>
<html lang="en">
    <head>
        <meta charset="UTF-8">
        <link rel="stylesheet" href="{{ url_for('static', filename= 'css/style.css') }}">
        <title>ML model API</title>
    </head>
    <body>
        <h1>API du modèle de classification binaire</h1>       
        <h2>Formulaire de saisie d'attributs</h2>       
        
        <form method="post" action="{{ url_for('result') }}">
            <div>
                <label for="feat1">Feature 1 :</label>
                <input name="umap_x" id="feat1" type="number" step="any" value="0.3">
            </div>
            <div>
                <label for="feat2">Feature 2 :</label>
                <input name="umap_y" id="feat2" type="number" step="any" value="0.3">
            </div>
            <div>
                <button>Predict</button>
            </div>
        </form>
      
    </body>
</html>



#### Résultat
Page de résultat de prédiction par l'API du modèle

In [9]:
file_html_result = dir_html + 'result.html'
!cat $file_html_result


<!DOCTYPE html>
<html lang="en">
    <head>
        <meta charset="UTF-8">
        <link rel="stylesheet" href="{{ url_for('static', filename= 'css/style.css') }}">
        <title>ML model API</title>
    </head>
    <body>
       <h1>API du modèle de classification binaire</h1>      
       <h2>Page de prédiction du modèle</h2> 
       <p>umap_x = {{features['umap_x']}}</p>
       <p>umap_y = {{features['umap_y']}}</p>
       <p>valeur prédite = {{target_value['predictions']}}</p>
       <a href="/">Retour à l'accueil</a>
       <a href="/form/">Formulaire</a>
    </body>
</html>



#### Style

In [10]:
dir_style = './static/css/'

In [11]:
file_style = dir_style + 'style.css'
!cat $file_style


h1 {
    border: 2px #eee solid;
    color: brown;
    text-align: center;
    padding: 10px;
}



### Fichiers _Python_

#### Routeur Web  
Un fichier en Pyhton couvre la gestion REST

In [23]:
file_api = 'web_router.py'
!cat $file_api


from flask import Flask, render_template, request
import requests
import json

app = Flask(__name__) # get work dirs

@app.route('/')                                           # index
def index():
    return render_template('index.html') # HTML as string

@app.route('/form/')                                      # form
def form():
    return render_template('form.html')

@app.route('/result/', methods=['POST'])                  # result predicted by API
def result():    
    dict_features = request.form.to_dict(flat=False) #{umap_x:[.3],umap_y:[.3]}
    req_post = requests.post(   url     = 'http://localhost:5678/invocations', 
                                headers = {'Content-Type': 'application/json'}, 
                                data    = json.dumps({'inputs': dict_features}) )
    dict_prediction = json.loads(req_post.text)     #{predictions:[1]}
    return render_template('result.html', features=dict_features, target_value=dict_prediction)

@app.route('/deploy/')          

### Arborescence de fichiers

In [24]:
! find ../website | sed -e "s/[^\/]*\//  |/g" -e "s/|\([^ ]\)/└── \1/"

  └── website
  |  └── .ipynb_checkpoints
  |  |  └── Jean_Vallée_5_notebook_test_API_042024-checkpoint.ipynb
  |  └── web_router.py
  |  └── templates
  |  |  └── result.html
  |  |  └── index.html
  |  |  └── form.html
  |  └── static
  |  |  └── css
  |  |  |  └── style.css
  |  └── Jean_Vallée_5_notebook_test_API_042024.ipynb


## Serveur _Web_

### Lancement

In [26]:
shell_command = 'flask --app $file_api run -h 0.0.0.0 -p 6543 --debug > ./flask_app.log 2>&1 &'
get_ipython().system_raw(shell_command) # run model API in background

### Vérifications
#### Log

In [27]:
!tail ./flask_app.log

 * Serving Flask app 'web_router.py'
 * Debug mode: on
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:6543
 * Running on http://10.0.0.4:6543
Press CTRL+C to quit
 * Restarting with stat
 * Debugger is active!
 * Debugger PIN: 134-700-135


#### Processus
Liste des processus

In [28]:
li_ps = !ps aux | grep "flask" | grep -v "grep" | awk '{print $2}' 
!ps aux | grep "flask" | grep -v "grep" 

azureus+  131413  1.0  0.9  37648 31896 ?        S    21:31   0:00 /home/azureuser/environments_folder/my_env/bin/python3 /home/azureuser/environments_folder/my_env/bin/flask --app web_router.py run -h 0.0.0.0 -p 6543 --debug
azureus+  131414  1.5  0.9 111380 31768 ?        Sl   21:31   0:00 /home/azureuser/environments_folder/my_env/bin/python3 /home/azureuser/environments_folder/my_env/bin/flask --app web_router.py run -h 0.0.0.0 -p 6543 --debug


Arrêt des processus

In [29]:
for ps_i in li_ps : 
    #!kill -9 $ps_i

SyntaxError: incomplete input (2464387668.py, line 2)

#### Accès en ligne
URL du site Web : http://13.92.86.145:6543/